In [1]:
import patchdata
import model
import test
import torch
import torch.optim as optim
import torch.nn as nn

if __name__ == "__main__":
    img_size = 32
    patch_size = 4
    batch_size = 128
    save_acc =50
    epochs = 500
    lr = 0.001
    drop_rate = .1
    weight_decay = 0
    num_classes = 10
    latent_vec_dim = 128
    num_heads = 8
    num_layers = 12
    dataname = 'cifar10'
    mode = 'train'
    pretrained = 0



    latent_vec_dim = latent_vec_dim
    mlp_hidden_dim = int(latent_vec_dim/2)
    num_patches = int((img_size * img_size) / (patch_size * patch_size))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Image Patches
    d = patchdata.Flattened2Dpatches(dataname=dataname, img_size=img_size, patch_size=patch_size,
                                     batch_size=batch_size)
    trainloader, valloader, testloader = d.patchdata()
    image_patches, _ = iter(trainloader).next()

    
    # Model
    vit = model.VisionTransformer(patch_vec_size=image_patches.size(2), num_patches=image_patches.size(1),
                                  latent_vec_dim=latent_vec_dim, num_heads=num_heads, mlp_hidden_dim=mlp_hidden_dim,
                                  drop_rate=drop_rate, num_layers=num_layers, num_classes=num_classes).to(device)

    if pretrained == 1:
        vit.load_state_dict(torch.load('./model.pth'))

    if mode == 'train':
        # Loss and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(vit.parameters(), lr=lr, weight_decay=weight_decay)

        #optimizer = torch.optim.SGD(vit.parameters(), lr=args.lr, momentum=0.9)
        #scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=args.lr, steps_per_epoch=len(trainloader), epochs=args.epochs)

        # Train
        n = len(trainloader)
        best_acc = save_acc
        for epoch in range(epochs):
            running_loss = 0
            for img, labels in trainloader:
                optimizer.zero_grad()
                outputs, _ = vit(img.to(device))
                loss = criterion(outputs, labels.to(device))
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                #scheduler.step()

            train_loss = running_loss / n
            val_acc, val_loss = test.accuracy(valloader, vit)
            # if epoch % 5 == 0:
            print('[%d] train loss: %.3f, validation loss: %.3f, validation acc %.2f %%' % (epoch, train_loss, val_loss, val_acc))

            if val_acc > best_acc:
                best_acc = val_acc
                # print('[%d] train loss: %.3f, validation acc %.2f - Save the best model' % (epoch, train_loss, val_acc))
                torch.save(vit.state_dict(), './model.pth')

    else:
        test_acc, test_loss = test.accuracy(testloader, vit)
        print('test loss: %.3f, test acc %.2f %%' % (test_loss, test_acc))

Files already downloaded and verified
Files already downloaded and verified
[0] train loss: 1.719, validation loss: 1.388, validation acc 50.92 %
[1] train loss: 1.393, validation loss: 1.267, validation acc 55.00 %
[2] train loss: 1.286, validation loss: 1.266, validation acc 55.32 %
[3] train loss: 1.208, validation loss: 1.134, validation acc 59.68 %
[4] train loss: 1.158, validation loss: 1.134, validation acc 58.90 %
[5] train loss: 1.118, validation loss: 1.055, validation acc 62.66 %
[6] train loss: 1.079, validation loss: 1.034, validation acc 63.08 %
[7] train loss: 1.048, validation loss: 1.031, validation acc 63.90 %
[8] train loss: 1.025, validation loss: 1.007, validation acc 64.52 %
[9] train loss: 0.996, validation loss: 0.963, validation acc 65.26 %
[10] train loss: 0.965, validation loss: 0.955, validation acc 66.72 %
[11] train loss: 0.954, validation loss: 0.958, validation acc 65.86 %
[12] train loss: 0.927, validation loss: 0.941, validation acc 66.50 %
[13] train 

KeyboardInterrupt: 